In [1]:
using Plots;plotly()
using DataFrames;using CSV
using Pkg
using LinearAlgebra
ccall((:openblas_get_num_threads64_, Base.libblas_name),Cint,())
LinearAlgebra.BLAS.set_num_threads(8)
using Base.Threads;nthreads()

1

In [2]:
function thermal_state(dims,nbar)
    c::Array{Complex{Float64}}=zeros(dims,dims)
    if nbar==0
        c[1,1]=1
    else
        for i in 1:dims
           c[i,i]= exp(-i*log(1/nbar+1))
        end
    end
    c=c/tr(c)
    return c        
end

function fock_state(dims,n)
    c::Matrix{ComplexF64}=zeros(dims,dims)
    if n<dims
        c[n+1,n+1]=1
    end
    return c
end

function eye(n)
    k::Array{Complex{Float64}}=zeros(n,n)
    for i in 1:n
        k[i,i]=1
    end
    return k    
end


function a₋(n)      #n为维数
    s::Array{Complex{Float64}}=zeros(n,n)
    for i=1:n-1
        s[i,i+1]=sqrt(i)
    end
    return s
end

a₊(n)=adjoint(a₋(n));

dim=10
ν = 2 * pi * 1.3
initial_state=kron([1.0 0 0 0;0 0 0 0;0 0 0 0;0 0 0 0],thermal_state(dim,1))

function dm2vec(dm)#态向量化
#julia里是按列reshape的,reshape成列向量
    return reshape(dm,:,1)
end

function vec2dm(vec)
    return reshape(vec,4*dim,:)
end

#按照g,r,e,t的顺序
n=kron(eye(4),a₊(dim)*a₋(dim))
ee=kron([0 0 0 0;0 0 0 0;0 0 1 0;0 0 0 0],eye(dim))
rr=kron([0 0 0 0;0 1 0 0;0 0 0 0;0 0 0 0],eye(dim))
tt=kron([0 0 0 0;0 0 0 0;0 0 0 0;0 0 0 1],eye(dim))

η=0.15
costheta = sqrt(2)/2
γg = 2 * pi * 6.89 
γr = 2 * pi * 13.79  

a=a₊(dim)+a₋(dim)
ge=kron([0 0 1 0;0 0 0 0;0 0 0 0;0 0 0 0],exp(im*η*costheta*a))
eg=kron([0 0 0 0;0 0 0 0;1 0 0 0;0 0 0 0],exp(-im*η*costheta*a))
u=ge+eg
rt=kron([0 0 0 0;0 0 0 1;0 0 0 0;0 0 0 0],exp(im*η*costheta*a))
t_r=kron([0 0 0 0;0 0 0 0;0 0 0 0;0 1 0 0],exp(-im*η*costheta*a))
u2=rt+t_r
uu=u+u2
r_e=kron([0 0 0 0;0 0 1 0;0 0 0 0;0 0 0 0],exp(-im*η*costheta*a))
er=kron([0 0 0 0;0 0 0 0;0 1 0 0;0 0 0 0],exp(im*η*costheta*a))
u3=r_e+er

E=eye(4*dim)

function diss1()
    dtheta = 0.01
    x=zeros((4*dim)^2,(4*dim)^2)
    for theta in -1:dtheta:1
        c = dtheta * 0.75 * (1 + theta^2)
        jump = sqrt(c / 2) * exp(im*η*theta*a)
        aup=kron([0 0 0 0;0 0 0 0;1 0 0 0;0 0 0 0],jump')
        adown=kron([0 0 1 0;0 0 0 0;0 0 0 0;0 0 0 0],jump)
        x=x+kron(transpose(aup),adown)-0.5*kron(E,aup*adown)-0.5*kron(transpose(aup*adown),E)
    end
    return x
end
function diss2()
    dtheta = 0.01
    x=zeros((4*dim)^2,(4*dim)^2)
    for theta in -1:dtheta:1
        c = dtheta * 0.75 * (1 + theta^2)
        jump = sqrt(c / 2) * exp(im*η*theta*a)
        aup=kron([0 0 0 0;0 0 0 0;0 0 0 0;0 1 0 0],jump')
        adown=kron([0 0 0 0;0 0 0 1;0 0 0 0;0 0 0 0],jump)
        x=x+kron(transpose(aup),adown)-0.5*kron(E,aup*adown)-0.5*kron(transpose(aup*adown),E)
    end
    return x
end
function diss3()
    dtheta = 0.01
    x=zeros((4*dim)^2,(4*dim)^2)
    for theta in -1:dtheta:1
        c = dtheta * 1.5 * (1 - theta^2)
        jump = sqrt(c / 2) * exp(im*η*theta*a)
        aup=kron([0 0 0 0;0 0 0 0;0 1 0 0;0 0 0 0],jump')
        adown=kron([0 0 0 0;0 0 1 0;0 0 0 0;0 0 0 0],jump)
        x=x+kron(transpose(aup),adown)-0.5*kron(E,aup*adown)-0.5*kron(transpose(aup*adown),E)
    end
    return x
end
function diss4()
    dtheta = 0.01
    x=zeros((4*dim)^2,(4*dim)^2)
    for theta in -1:dtheta:1
        c = dtheta * 1.5 * (1 - theta^2)
        jump = sqrt(c / 2) * exp(im*η*theta*a)
        aup=kron([0 0 0 0;0 0 0 0;0 0 0 0;1 0 0 0],jump')
        adown=kron([0 0 0 1;0 0 0 0;0 0 0 0;0 0 0 0],jump)
        x=x+kron(transpose(aup),adown)-0.5*kron(E,aup*adown)-0.5*kron(transpose(aup*adown),E)
    end
    return x
end
dissipation1=γg*diss1()
dissipation2=γr*diss2()
dissipation3=γr*diss3()
dissipation4=γg*diss4()
instate=dm2vec(initial_state);

In [3]:
function eit_Lindbladian_vec(ΩL1,ΩL2,Δg,Δr)
    unitary::Matrix{ComplexF64}=kron(E,-im*(ν*n-Δg*ee+(Δr-Δg)*rr+(Δr-Δg-(Δg+8*ν))*tt+ΩL1/2*uu+ΩL2/2*u3))+kron(transpose(im*(ν*n-Δg*ee+(Δr-Δg)*rr+(Δr-Δg-(Δg+8*ν))*tt+ΩL1/2*uu+ΩL2/2*u3)),E)
    
    return unitary+dissipation1+dissipation2+dissipation3+dissipation4
end

eit_Lindbladian_vec (generic function with 1 method)

In [16]:
# initial：
# 14.049202346853557, 148.91149178015618, 678.5840131753953, 678.5840131753953

# 40:
#  29.414155980164807, 143.02321686599555, 562.7272532543809, 561.1661331065062

# 90:
#  22.22895126792895, 182.74814937263437, 921.73389263177, 920.6575268329686

# 150:
#  18.39374569668173, 218.65695647321104, 1325.0343585188957, 1324.110515444204

# 250:
#  15.061759135858427, 265.08526852940776, 1953.720834601189, 1952.876854452561

In [ ]:
LinearAlgebra.BLAS.set_num_threads(1)
function nt()
    ttot=150
    space=3
    s_omega_g=1:space: 301
    s_omega_r=1:space: 301
    nf = zeros(length(s_omega_g),length(s_omega_r))
    datane=DataFrame(omega_g=s_omega_g,omega_r=s_omega_r)
    @threads for omegag in s_omega_g
        for omegar in s_omega_r
            nf[convert(Int64,round((omegag-s_omega_g[1])/space+1)),convert(Int64,round((omegar-s_omega_r[1])/space+1))]=real(tr(n*vec2dm(exp(ttot*eit_Lindbladian_vec(omegag,omegar, 1325.0343585188957, 1324.110515444204))*instate)))
        end
        print(omegag," ")
    end
    datane=hcat(datane,DataFrame(nf,:auto))
    CSV.write("omegag_omegar_sweep150.csv", datane)
    return nf
end
nt()

In [6]:
data3d=CSV.read("omegag_omegar_sweep150.csv", DataFrame)
nsurf=Array(data3d[:,Between(:x1, end)])
surface(data3d.omega_g,data3d.omega_r,nsurf,xlabel="omega_r",ylabel="omega_g",zlabel="n_t",xticks=0:50:305,yticks=0:50:305)

<!DOCTYPE html>
 
 
 Plots.jl